In [1]:
import os
import geopandas as gpd
import pandas as pd

In [2]:
import sys
sys.path.append('../')
from process.append import get_enriched_features

D:\WORK\wildfire\ITS_RECODE\interagency-tracking-system\notebooks\..\process\append.py:107: SyntaxWarning: invalid escape sequence '\c'
  with open("..\config.yaml", 'r') as stream:


In [3]:
california_boundary = gpd.read_file(r'D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb', 
                                    driver='OpenFileGDB', 
                                    layer='California')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [3]:
enriched_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1"

In [4]:
enriched_list = os.listdir(enriched_path)

In [5]:
append_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\appended.gdb"

In [6]:
report_path = r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\reports.gdb'

In [7]:
# skip the append, reports gdb file path
skip_list = ['appended.gdb', 'reports.gdb']

In [9]:
point_layers = []
line_layers = []
poly_layers = []
for f in enriched_list:
    if f in skip_list:
        continue
    f_path = enriched_path + r"\{}".format(f)
    gdb_layers = gpd.list_layers(f_path)
    for i in range(len(gdb_layers)):
        if 'point' in gdb_layers.loc[i, 'geometry_type'].lower():
            point_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        elif 'line' in gdb_layers.loc[i, 'geometry_type'].lower():
            line_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        else:
            poly_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
            
enriched_layers = {'point': point_layers, 
                  'line': line_layers,
                  'polygon': poly_layers}

In [10]:
# one time change for counts to mas

In [10]:
enriched_layers

{'point': [{'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0.1\\CNRA_1950_2025.gdb',
   'layer_name': 'CNRA_enriched_20250814_point'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0.1\\NFPORS_1950_2025.gdb',
   'layer_name': 'NFPORS__20250814_point'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0.1\\PFIRS_1950_2025.gdb',
   'layer_name': 'PFIRS_20250814'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0.1\\Timber_Nonspatial_1950_2025.gdb',
   'layer_name': 'Timber_Nonspatial_20250814'}],
 'line': [{'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0.1\\CalTRANS_1950_2025.gdb',
   'layer_name': 'CalTRANS_enriched_20250814'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0.1\\CNRA_1950_2025.gdb',
   'layer_name': 'CNRA_enriched_20250814_line'}],
 '

In [12]:
ifprs_dict = {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0.1\\IFPRS_2023_2025.gdb',
   'layer_name': 'IFPRS_enriched_20250809'}
usfs_dict = [{'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0.1\\USFS_1950_2025.gdb',
   'layer_name': 'USFS_Region04_enriched_20250809'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0.1\\USFS_1950_2025.gdb',
   'layer_name': 'USFS_Region05_enriched_20250809'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0.1\\USFS_1950_2025.gdb',
   'layer_name': 'USFS_Region06_enriched_20250809'}]

In [19]:
from utils.category import categorize_activity
from utils.standardize_domains import standardize_domains
from utils.counts_to_mas import counts_to_mas

# semi-reprocess ifprs and usfs from Kai

In [127]:
ifprs_gdf = gpd.read_file(ifprs_dict['gdb_path'], driver='OpenFileGDB', layer=ifprs_dict['layer_name'])

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [129]:
ifprs_gdf.ADMINISTERING_ORG.unique()

array(['FWS', 'NPS', 'BIA'], dtype=object)

In [88]:
for lyr_type in enriched_layers.keys():
    for path_dict in enriched_layers[lyr_type]:
        gdf = gpd.read_file(path_dict['gdb_path'], driver='OpenFileGDB', layer=path_dict['layer_name'])
        gdf = counts_to_mas(standardize_domains(categorize_activity(gdf)), 1950, 2025)
        #gdf = counts_to_mas(gdf, 1950, 2025)
        #if ('BLM' in path_dict['layer_name']) or ('NFPORS' in path_dict['layer_name']) or ('NPS' in path_dict['layer_name']):
        #    print(path_dict['layer_name'])
        #    gdf.loc[gdf['ACTIVITY_START'] >= f'2023-10-01', 'COUNTS_TO_MAS'] = 'NO'
        print(path_dict['layer_name'])
        gdf.to_file(path_dict['gdb_path'], driver='OpenFileGDB', layer=path_dict['layer_name'])
    

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-08-14 18:06:19,866 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-08-14 18:06:19,867 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-08-14 18:06:19,867 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-08-14 18:06:19,869 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-08-14 18:06:19,869 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-08-14 18:06:19,870 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-08-14 18:06:19,870 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category i

NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
None
NOT_DEFINED
None
OTHER_FUELS_REDUCTION
None
OTHER_FUELS_REDUCTION
None
FOREST_STEWARDSHIP
None
OTHER_FOREST_MGMT
None
OTHER_FOREST_MGMT
CNRA_enriched_20250814_point


2025-08-14 18:06:20,033 INFO  [pyogrio._io           ]  Created 2,066 records
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-08-14 18:06:20,390 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-08-14 18:06:20,391 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-08-14 18:06:20,392 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-08-14 18:06:20,395 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-08-14 18:06:20,396 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-08-14 18:06:20,397 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-08-14 18:06:20,398 INFO  [utils.coun

NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
None
None
None
None
None
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
NOT_DEFINED
OTHER_FUELS_REDUCTION
None
None
None
None
None
None
None
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
None
None
None
None
None
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
None
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
None
None
None
None
None
None
None
None
None
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
NOT_DEFINED
NOT_DEFINED
None
None
None
None
None
None
None
None
None
None
NOT_DEFINED
NOT_DEFINED
None
None
None
None
None
None
None
None
NOT_DEFINED
NO

2025-08-14 18:06:20,656 INFO  [pyogrio._io           ]  Created 4,555 records
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-08-14 18:06:21,696 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-08-14 18:06:21,696 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-08-14 18:06:21,697 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-08-14 18:06:21,702 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-08-14 18:06:21,704 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-08-14 18:06:21,706 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-08-14 18:06:21,708 INFO  [utils.coun

...

FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD


2025-08-14 18:08:00,519 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-08-14 18:08:00,520 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-08-14 18:08:00,525 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-08-14 18:08:00,542 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-08-14 18:08:00,590 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-08-14 18:08:00,633 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-08-14 18:08:00,682 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category is 'Watershed Improvement'
2025-08-14 18:08:00,701 INFO  [utils.counts_to_mas   ]              counts step 7/8: set to 'NO' if Agency is 'Other' and Admin is 'CARB'


USFS_Region05_enriched_20250809


2025-08-14 18:08:20,934 INFO  [pyogrio._io           ]  Created 412,493 records
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-08-14 18:08:27,289 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-08-14 18:08:27,290 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-08-14 18:08:27,290 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-08-14 18:08:27,291 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-08-14 18:08:27,291 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-08-14 18:08:27,291 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-08-14 18:08:27,292 INFO  [utils.co

NOT_DEFINED
REFORESTATION
TBD
TBD
TBD
TBD
NOT_DEFINED
REFORESTATION
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
USFS_Region06_enriched_20250809


In [89]:
enriched_polygons, enriched_lines, enriched_points = get_enriched_features(enriched_layers)

2025-08-14 18:08:27,407 INFO  [process.append_polygon]  --------------------------------------------------------------------------------
2025-08-14 18:08:27,407 INFO  [process.append_polygon]  Concatenate all polygon records
2025-08-14 18:08:27,408 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'BLM_enriched_20250710' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\BLM_1950_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-08-14 18:08:27,529 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'CNRA_enriched_20250814_polygon' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\CNRA_1950_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-08-14 18:08:31,504

In [29]:
import dask_geopandas
# some data need to be converted to multi-type again
from utils.save_gdf_to_gdb import save_gdf_to_gdb

In [91]:
# grab timber non spatial path again
timber_nonspatial_path = None
for p in enriched_layers['point']:
    if 'Timber_Nonspatial' in p['gdb_path']:
        timber_nonspatial_path = p
        break

In [92]:
timber_nonspatial = gpd.read_file(timber_nonspatial_path['gdb_path'], 
                                    driver='OpenFileGDB', 
                                    layer=timber_nonspatial_path['layer_name'])

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [93]:
%%time
for df, lyr_name in zip([enriched_polygons,enriched_lines,enriched_points], ["appended_poly","appended_line","appended_point"]):
    # init dask gdf for multithread clipping
    ddf = dask_geopandas.from_geopandas(df, npartitions=16)
    # clip to california bounds
    append_clipped = ddf.sjoin(california_boundary, how='inner', predicate='intersects').compute()
    # drop unwanted artifact columns from California boundary df
    append_clipped = append_clipped.drop(['index_right', 'Shape_Area', 'Shape_Length'], axis=1)
    
    # industry nonspatial is by design out of california bounds and got clipped, manually concat it back
    if lyr_name == 'appended_point':
        append_clipped = pd.concat([append_clipped, timber_nonspatial], ignore_index=True)
    save_gdf_to_gdb(append_clipped, append_path, lyr_name)

2025-08-14 18:26:25,653 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-08-14 18:26:25,653 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-08-14 18:26:25,683 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\appended.gdb


2025-08-14 18:26:58,470 INFO  [pyogrio._io           ]  Created 454,597 records
2025-08-14 18:27:06,339 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\appended.gdb appended_poly
2025-08-14 18:33:04,056 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-08-14 18:33:04,056 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-08-14 18:33:04,066 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\appended.gdb


2025-08-14 18:33:27,568 INFO  [pyogrio._io           ]  Created 131,658 records
2025-08-14 18:33:35,865 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\appended.gdb appended_line
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a futu

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\appended.gdb


2025-08-14 18:33:39,012 INFO  [pyogrio._io           ]  Created 20,443 records
2025-08-14 18:33:39,158 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\appended.gdb appended_point


CPU times: total: 52min 32s
Wall time: 24min 26s


In [94]:
gpd.list_layers(append_path)

,name,geometry_type
0,appended_poly,MultiPolygon
1,appended_line,MultiLineString
2,appended_point,Point


In [8]:
upload_path = r"D:/WORK/wildfire/Interagency-Tracking-System/its/ITSGDB_backup/V2.0upload"
enriched_points = gpd.read_file(os.path.join(upload_path,'ITS_V2_0_Points.gdb'), driver='OpenFileGDB', layer='ITS_V2_0_Points')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [9]:
enriched_polygons = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_poly')
#enriched_points = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_point')
enriched_lines = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_line')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [10]:
enriched_points[(enriched_points.AGENCY =='<NA>')].PROJECTID_USER.apply(lambda x: x[:5]).unique()

array([], dtype=object)

In [11]:
enriched_polygons[(enriched_polygons.COUNTS_TO_MAS=="YES") & (enriched_polygons.ADMINISTERING_ORG.isna())]

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,...,Federal_FY,State_FY,TRMT_GEOM,COUNTS_TO_MAS,FIPS,STATE,ST_ABBREV,SQMI,POPULATION,geometry


In [12]:
enriched_polygons[(enriched_polygons.COUNTS_TO_MAS == 'YES') & (enriched_polygons.ACTIVITY_STATUS == 'ACTIVE')].AGENCY.unique()

array(['CNRA'], dtype=object)

In [13]:
enriched_polygons.AGENCY.unique()

array(['DOI', 'CNRA', 'OTHER', 'FWS', 'NPS', 'BIA', 'TIMBER', 'USDA'],
      dtype=object)

In [14]:
enriched_points.AGENCY.unique()

array(['CNRA', 'DOI', 'OTHER', 'PARKS', 'TIMBER', 'DOD', 'DOE'],
      dtype=object)

In [15]:
enriched_points.REGION.unique()

array(['SOUTHERN_CA', 'CENTRAL_COAST', 'SIERRA_NEVADA', 'NORTH_COAST',
       'Private – Industrial'], dtype=object)

In [16]:
enriched_lines.AGENCY.unique()

array(['CALSTA', 'CNRA'], dtype=object)

In [17]:
enriched_polygons[enriched_polygons.ACTIVITY_CAT.isna()]

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,...,Federal_FY,State_FY,TRMT_GEOM,COUNTS_TO_MAS,FIPS,STATE,ST_ABBREV,SQMI,POPULATION,geometry


In [18]:
enriched_lines[enriched_lines.ACTIVITY_CAT.isna()]

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,...,Federal_FY,State_FY,TRMT_GEOM,COUNTS_TO_MAS,FIPS,STATE,ST_ABBREV,SQMI,POPULATION,geometry


In [19]:
enriched_points[enriched_points.ACTIVITY_CAT.isna()]

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,...,Federal_FY,State_FY,TRMT_GEOM,COUNTS_TO_MAS,FIPS,STATE,ST_ABBREV,SQMI,POPULATION,geometry


In [20]:
enriched_points[enriched_points.ACTIVITY_CAT == 'NOT_DEFINED'].ACTIVITY_DESCRIPTION.unique()

array(['NOT_DEFINED', 'TBD'], dtype=object)

In [21]:
enriched_lines[enriched_lines.ACTIVITY_CAT == 'NOT_DEFINED'].ACTIVITY_DESCRIPTION.unique()

array(['TBD', 'NOT_DEFINED'], dtype=object)

In [22]:
enriched_polygons[enriched_polygons.ACTIVITY_CAT == 'NOT_DEFINED'].ACTIVITY_DESCRIPTION.unique()

array(['TBD', 'NOT_DEFINED'], dtype=object)

In [23]:
#excel_out = pd.concat([enriched_points, enriched_lines, enriched_polygons])
#excel_out.drop(['geometry'], axis=1).to_csv('its_all.csv')

In [24]:
from datetime import datetime

In [25]:
def get_activity_report(enriched_points, enriched_lines, enriched_polygons):
    append_all = pd.concat([enriched_lines, enriched_points, enriched_polygons])
    append_all = append_all[(append_all['COUNTS_TO_MAS'] == 'YES')]
    
    # 
    append_all.geometry = gpd.points_from_xy(append_all['LONGITUDE'],append_all['LATITUDE'])
    
    
    append_all = append_all[["AGENCY",
        "ADMINISTERING_ORG",
        "PRIMARY_OWNERSHIP_GROUP",
        "COUNTY",
        "REGION",
        "ACTIVITY_DESCRIPTION",
        "ACTIVITY_CAT",
        "BROAD_VEGETATION_TYPE",
        "ACTIVITY_STATUS",
        "ACTIVITY_QUANTITY",
        "ACTIVITY_UOM",
        "ACTIVITY_END",
        "Year_txt",
        "geometry"
    ]]
    
    # check if geometry is_valid
    # personally this is redundent, unless lat, lon is not valid, but that would throw an error in previous part
    append_all = append_all[append_all.is_valid]
    
    
    def get_entity_type(agency):
        if agency in ['CALEPA', 'CALSTA', 'CNRA', 'PARKS', 'California State Parks']:
            return 'State'
        if agency in ['DOD', 'DOI', 'USDA', 'DOE', 'NPS']:
            return 'Federal'
        if agency in ['Industrial Timber', 'Timber Companies', 'TIMBER']:
            return 'Timber Companies'
        else:
            return None
        
        
    append_all['ENTITY_TYPE'] = append_all['AGENCY'].apply(get_entity_type)
    
    return append_all

In [26]:
activity_report_gdf = get_activity_report(enriched_points, enriched_lines, enriched_polygons)

In [30]:
save_gdf_to_gdb(activity_report_gdf, 
                os.path.join(upload_path, 'ITSV2.0_Activity.gdb'), 
                'ITSV2.0_Activity')

2025-08-22 00:23:11,897 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-08-22 00:23:11,898 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-08-22 00:23:11,924 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:/WORK/wildfire/Interagency-Tracking-System/its/ITSGDB_backup/V2.0upload\ITSV2.0_Activity.gdb


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered layer name: 'ITSV2.0_Activity' to 'ITSV2_0_Activity'
  ogr_write(
2025-08-22 00:23:18,290 INFO  [pyogrio._io           ]  Created 494,926 records
2025-08-22 00:23:21,239 INFO  [utils.save_gdf_to_gdb ]        File saved to D:/WORK/wildfire/Interagency-Tracking-System/its/ITSGDB_backup/V2.0upload\ITSV2.0_Activity.gdb ITSV2.0_Activity


In [111]:
save_gdf_to_gdb(activity_report_gdf, 
                report_path, 
                f'activity_report{datetime.today().strftime('%Y%m%d')}')

2025-08-14 18:34:48,980 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-08-14 18:34:48,980 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-08-14 18:34:49,010 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\reports.gdb


2025-08-14 18:34:55,309 INFO  [pyogrio._io           ]  Created 494,926 records
2025-08-14 18:34:58,227 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\reports.gdb activity_report20250814


In [ ]:
activity_report_gdf = gpd.read_file(report_path, driver='OpenFileGDB', layer='activity_report20250729')

In [31]:
activity_report_gdf.ACTIVITY_STATUS.unique()

array(['COMPLETE', 'ACTIVE'], dtype=object)

In [32]:
activity_report_gdf[activity_report_gdf.ACTIVITY_CAT.isna()].AGENCY.unique()

array([], dtype=object)

In [33]:
activity_report_gdf[activity_report_gdf.Year_txt == '2024'].groupby(['AGENCY', 'ADMINISTERING_ORG', 'ACTIVITY_STATUS']).sum('ACTIVITY_QUANTITY')

ACTIVITY_QUANTITY
AGENCY ADMINISTERING_ORG ACTIVITY_STATUS                   
BIA    BIA               COMPLETE                 69.000000
CALSTA CALTRANS          COMPLETE             133459.499800
CNRA   CALFIRE           ACTIVE               138469.461500
                         COMPLETE              33542.742500
       CCC               ACTIVE                  261.190000
                         COMPLETE               1522.270000
       CDFW              COMPLETE             103877.059607
       PARKS             COMPLETE              15087.253321
       SDRC              ACTIVE                 3842.530000
                         COMPLETE                131.160000
       SMMC              COMPLETE                618.066657
       SNC               ACTIVE                10441.770000
                         COMPLETE               1802.690000
       TAHOE             ACTIVE                  250.500000
                         COMPLETE                 54.000000
       WCB               ACTIVE                 1301.550000
                         COMPLETE               6369.910000
DOD    DOD               COMPLETE               2967.330000
DOE    DOE               COMPLETE               5482.900000
DOI    BLM               COMPLETE              40467.823381
FWS    FWS               COMPLETE              19255.000000
NPS    NPS               COMPLETE               3547.000000
TIMBER TIMBER            COMPLETE             186690.412376
USDA   USFS              COMPLETE             290739.200000

In [126]:
enriched_polygons.loc[enriched_polygons.AGENCY.isin(['FWS', 'BIA', 'NPS'])]

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,...,Federal_FY,State_FY,TRMT_GEOM,COUNTS_TO_MAS,FIPS,STATE,ST_ABBREV,SQMI,POPULATION,geometry
35877,CADBR-HFR-FY24-MX-W,FWS,FWS,None,None,FWS,CADBR-HFR-FY24-MX-W,None,None,None,...,2024,2024,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((273163.046 -598315.334, 273154..."
35878,CAR8R-BIL-FY23-MX-San Diego,FWS,FWS,None,None,FWS,CAR8R-BIL-FY23-MX-San Diego,None,None,None,...,2024,2024,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((273495.062 -597705.931, 273500..."
35880,CATNR-HFR-FY24-MX-W,FWS,FWS,None,None,FWS,CATNR-HFR-FY24-MX-W,None,None,None,...,2024,2025,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((294989.891 -581637.748, 294979..."
35881,CASOR-HFR-FY24-RX-N,FWS,FWS,None,None,FWS,CASOR-HFR-FY24-RX-N,None,None,None,...,2024,2024,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((412018.092 -526655.842, 412041..."
35882,CASOR-HFR-FY24-MX-N,FWS,FWS,None,None,FWS,CASOR-HFR-FY24-MX-N,None,None,None,...,2024,2025,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((411839.652 -526745.964, 411831..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36034,CATKR-HFR-FY24-CX-W,FWS,FWS,None,None,FWS,CATKR-HFR-FY24-CX-W,None,None,None,...,2024,2025,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((-129980.427 435082.319, -12998..."
36035,CAHBR-HFR-FY24-CX-W,FWS,FWS,None,None,FWS,CAHBR-HFR-FY24-CX-W,None,None,None,...,2024,2024,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((-348624.784 328880.86, -348630..."
36038,CALKR-HFR-FY24-MX-W,FWS,FWS,None,None,FWS,CALKR-HFR-FY24-MX-W,None,None,None,...,2024,2024,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((-139124.742 440043.332, -13912..."
36039,CALKR-HFR-FY24-CX-W,FWS,FWS,None,None,FWS,CALKR-HFR-FY24-CX-W,None,None,None,...,2024,2025,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((-147598.282 443459.815, -14756..."


In [34]:
activity_report_gdf[(activity_report_gdf.ACTIVITY_DESCRIPTION == "AMW_AREA_RESTOR") | (activity_report_gdf.ACTIVITY_DESCRIPTION == "OAK_WDLND_MGMT")].ACTIVITY_CAT.unique()

array(['MECH_HFR'], dtype=object)